# Worklog

## 2019-04-03 to 2019-04-05

Rereading lecture slides from Language Technology [EDAN20](http://cs.lth.se/edan20/lectures/).

Looking articles/websites/packages:

- [Semantic annotation in biomedicine: the current landscape](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5610427/)
- [A Survey of Bootstrapping Techniques in Natural Language Processing](https://www.eecis.udel.edu/~vijay/fall13/snlp/lit-survey/Bootstrapping.pdf)
- [POS-tagger and NER for bio-medical abstracts](http://www.nactem.ac.uk/tsujii/GENIA/tagger/)
    - [Developing a Robust Part-of-Speech Tagger for Biomedical Text]()


## 2019-04-10

Vilhelm is sick, so Olof and Vilhelm talk over VOIP and are installing software so they can start working.

Discussion how to approaching solving detecting relationships - _A inhibits B_. Come up with this plan:

1. Use Genia parser to generate Part-Of-Speech tags for all words in the abstracts.
    - GENIA parser was created for this task specifically: it's based on a feature set tailored to working on regular text and biomedical abstracts. Was then trained on a regular corpus as well as biomedical corpora.
1. Use the Stanford Core NLP package to generate dependency graphs
1. Use dependency graph to find the relationships.
    - Not sure how to do this yet, but we believe this should be possible. We think the head of the relevant sentences will be _"inhibits"_ or similar, and then the inhibitor and inhibitee will be directly related to it in the depency graph. 

### Potential problems

-  Generating POS's and a dependency graph for each abstract in the database might take an extremely long time. We might be able to get around this by filtering sentences
- Pierre said two different NLP packages might not always work well together. Hopefully this is just because of a differing POS scheme so we can add a POS-scheme translator between the POS tagger and the Dependency parser.
    - Apparently the Stanford CoreNLP package [can take annotated data](https://stanfordnlp.github.io/CoreNLP/cmdline.html#inputting-serialized-files), so this approach might still be viable.
    - We should maybe run a POS annotation test and compare if the tags are similar?


### Articles found

- [NLP-progress: Relationship Extraction](http://nlpprogress.com/english/relationship_extraction.html)
    - Might be useful for detecting relationships from dependency graphs?

## 2019-04-11

### Vilhelm
Created script to extract abstracts from the Pubmed XML files. The PMID of each article is attached in the generated output to enable easily finding the original article. Using NLTK to split abstracts into one sentence per line as this is necessary for GENIA Parser.
### Olof

Extracted abstracts from the XML-files, created dictionaries of PMID to abstracts, pipe reading to core, and produce dependencies/pos/parseQuestions: Faster using batches? Crashes when using batches of 70> abstracts. One of the example files had only 248 abstracts, the other one has ~25k.

## 2019-04-12

### Olof

Began extraction of dependencies from keywords. identify which dependency relations are relevant (nsubj dobj? talk to pierre)
### Vilhelm
Stanford Core NLP recommended to run their toolkit in batches instead of individual sentences, as it has some loading time.

Olof only seemed to be able to run the Stanford Core NLP in smaller batches, so I cleaned up the abstract extractor and added batch functionality.

Got a script from Olof regarding running and using Stanford Core NLP and tweaked it to take random batches of abstracts and run POS-tagging and Dependency parsing on them.

- Dependency-graph head detection seems not to be working right. From a biomedical abstract with 6 sentences (PMID: 5859018), the dependency parser registered 6 heads, all of them in the first sentence.
- POS-tagging with Stanford Core NLP seems to be working alright. From the same abstract we retrieved the following tags. I don't know much about medical terms, but I guess this looks right?

```
The	DT
incorporation	NN
of	IN
tritiated	JJ
thymidine	NN
into	IN
the	DT
DNA	NN
of	IN
erythroblasts	NNS
,	,
primitive	JJ
ependymal	JJ
cells	NNS
,	,
and	CC
mesenchymal	JJ
cells	NNS
of	IN
.....```

- The XML parser retrieved som weird stuff from _pubmed19n0195.xml_, one abstract had an author list and citations in it, which messed up the sentence splitter from NLTK, and thus messed up stuff for Stanford Core NLP.

## 2019-04-14

### Olof

can now retrieve both nsubj and dobj, including adjectives and nmodsHow to handle situations around 'it' or 'that'? Also, what are appos, nsubjpass etc?

## 2019-04-15

### Vilhelm

Had meeting with Sonja, Marcus, Anna and Eric.

Discussed project status.

We now understand better the work we are supposed to be doing: we assume that we have two known mentions/entities in a sentence, and our job is to find the relation between the two entites, if there is one.

Marcus showed us the [BioInfer corpus](http://mars.cs.utu.fi/BioInfer/?q=welcome) with has gold-standard entity-relationship annotation. Me and Olof will look into this next time we work.

## 2019-04-16

### Vilhelm

Created parser that scrapes sentences and their tokens, entites, and dependencies from the Bioinfer corpus.

#### Issues

- The dependency graph tags (deprels) have weird < and > characters in them. Don't know what that's about.
- What are the "parallel" dependencies in the corpus? They have a very different tagset.
### Olof

It took some time to understand BioInfer and how it was used. Used the scraper Vilhelm wrote, applied it to a simple machine learning model with multiple inputs. Need further help from Markus/Pierre. 

## 2019-04-18

### Vilhelm

Read more into the [article describing the corpus](https://bmcbioinformatics.biomedcentral.com/track/pdf/10.1186/1471-2105-8-50), and more interestingly into the [one concerning the Stanford Scheme update](http://mars.cs.utu.fi/BioInfer/files/pyysalo_et_al_on_the_unification.pdf), and found out that the "parallell" dependencies noted above is a Link Grammar annotation. Their update added dependency annotation using the [original Stanford Scheme](https://nlp.stanford.edu/software/dependencies_manual.pdf), and the < and > arrows represent the direction of the arrow in the dependency graph. Added an update to the BioInfer parser accounting for this.

I added some very simple relationship parsing to the BioInfer parser. In the _"formulas"_ section of a sentence in the corpus, they added the relationships between entities. The relationships exisiting in the sentences of the corpus are complex, for example:
    
    _alpha-catenin inhibits beta-catenin signaling by preventing formation of a beta-catenin*T-cell factor*DNA complex_
    
Two of the entities are _alpha-catenin_ and _beta_catenin signaling_, but _alpha-catenin_ does not _directly_ inhibit _beta-catenin signaling_. The corpus has multilayered  relationships to handle this.

For the time being, we ignore all these complex relations and try to only look at _A_ inhibits _B_ relationships, so we can move on to more advanced relations later.

#### Issues

- The dependencies are in the [original Stanford Scheme](https://nlp.stanford.edu/software/dependencies_manual.pdf). Stanford Core NLP still supports this scheme, but this might cause trouble. Hopefully there's some sort of scheme translator?
- The relationships we do get from parsing are absurd and not useful at all. Some examples:

```
alpha-catenin inhibits beta-catenin signaling by preventing formation of a beta-catenin*T-cell factor*DNA complex.

Relations:
  alpha-catenin [0]       NEGATIVE(inhibits)      preventing [5]

---------------------------------------------------------------------------------

We conclude that Aip1p is a cofilin-associated protein that enhances the filament disassembly activity of cofilin and restricts cofilin localization to cortical actin patches.

Relations:
  We [0]        POSITIVE(enhances)      that [2]

---------------------------------------------------------------------------------

We have previously shown that the synthetic dodecapeptide corresponding to Trp104-Met115 of cofilin is a potent inhibitor of actin polymerization.

Relations:
  We [0]        NEGATIVE(inhibitorof)   previously [2]
```

## 2019-04-22 to 2019-04-23

### Vilhelm

Fixed the previous issue with the weird relationships, it was a small programming error. Read up on Olofs grapher.py and trying to just get an overview of that, as it is based on techniques Olof learned from EDAN95.

Olof is trying to get his machine-learning algorithm to work, but needs help from Marcus Klang. As we do not want to start working on complex relations (described above in worklog for 2019-04-18) I won't be doing much in the meanwhile.

## 2019-05-03 to 2019-05-05

### Vilhelm

Olof had exams last week so we haven't had time to work together on the project really.

I've been putting together a GUI so we can visualize the relations (in **gui.py**, it's pretty much reached MVP I think), and Olof's been looking into Scispacy. He had multiple issues with needing to install things though, so we haven't made any concrete progress.
### Olof

Installed SpaCy and tried to get it to run on my graphics card instead of the CPU. SpaCy has plenty of nice-to-have features which helps with the relation extraction, but the extractor is not finished yet. 

## 2019-05-07 to 2019-05-08

### Vilhelm

Realized we need an easy way to represent entities and relations to see them in the GUI. I've been working at setting up that model and integrating it with the GUI. (Otherwise, creating all the dicts and lists to send into the gui will be real weird and hard to understand). Packaging actual data and integrating it into the GUI should now be pretty smooth, hopefully.

## 2019-05-09 - 2015-05-10

### Vilhelm

Modified the rule-based entity-relation detector (_spacynlp.py_) to be able to save character indices. Used this to connect it with the GUI using the Entity and Relation models. We miiiight have to add support to highlight several words individually as part of one entity/relation.

For example in the sentence _"The __president George Bush's__ (the latter) __golf swing__ was shown to be an __inducer__ of __laughter__."_, the first entity is __president George Bush's golf swing__.

## 2019-05-13

### Vilhelm

Added support for what was mentioned above. Added dynamic filtering of relation options based on the selected word. Changed so only entities _causing_ relations to appear in the top box. I.e. from a text "A inhibts B", only A will appear in the top box.

Moving forward we probably need to work more on coreference handling. For example in "We have found a cool protein. It prevents cancer." We need to make sure we find _cool protein_ instead of _it_. It sort of works now, but our script also collects prepositions to entities, and we need to marry prepositions and coreferences smartly.


## 2019-05-21

#### Issues:
 We noted some issues when going through a file received from Anna and Eric (_pubmed19n0001_).

- There are many first words that are unnecessary in our entity detection. For example in "___the__ X inhibited Y_", __the__ is included in the entity name ('_the X_') from our preposition collection. Also in stuff like "_our results indicate __that__ X inhibits y_".
- Coreferencing is probably quite needed. Our algorithm found 8 _'it'_ and 4 _'It'_ relations, as well as other coreferences like _"the enzyme"_.
- Statements of no relation are probably not handled correctly. In article PMID=25117, the sentence _"... since LSD did not increase the DOPA accumulation..."_, our algorithm detects the relation _"[LSD] - increase - [the DOPA accumulation]_".
- In article PMID=27595 with the sentence _" It, however, inhibited neither pepsin nor pronase."_, our algorithm detects the relation _"[It] - inhibited - [neither pepsin]"_. Here, _"nor pronase"_ is not registered.

#### Ideas:

- GUI: A search function to search through all the entities would be useful.
    - This would be most useful with a synonym dictionary: if the user searches for a protein that has an alternative name, entities containing that alternative name should also appear.
